In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import os
load_dotenv()

True

In [9]:
from pydantic import BaseModel, JsonValue

class GenericJSON(BaseModel):
    data: JsonValue


In [10]:
from __future__ import annotations
from typing import List, Dict, Optional, Any, Literal
from pydantic import BaseModel, JsonValue
from enum import Enum

# ---------------------------------------
# Primitive attribute/value types
# ---------------------------------------

class DataBinding(BaseModel):
    """
    Optional binding between component props and a data source.
    Example:
        {"source": "userData", "path": "stats.score"}
    """
    source: str               # name of dataset provided externally
    path: Optional[str] = ""  # dotted path inside the dataset


AttributeValue = JsonValue  # JSON-safe; allows str, number, bool, null, list, dict


# ---------------------------------------
# Core Component Schema
# ---------------------------------------

class Component(BaseModel):
    """
    Generic UI component definition.
    Works for both HTML tags (e.g., 'div', 'p') and custom components (e.g. 'Card', 'ChartBar').
    """
    type: str                                # e.g., "div", "button", "Card", "Chart"
    # props: Dict[str, AttributeValue] = {}    # HTML attributes or component props
    # style: Dict[str, AttributeValue] = {}    # Inline CSS-like style dictionary
    # events: Dict[str, str] = {}              # e.g. {"onClick": "handleClick"}
    # bind: Optional[Dict[str, DataBinding]] = None  # e.g. {"value": DataBinding(...)}
    children: Optional[List["Component"]] = None   # nested components


# ---------------------------------------
# Top-Level SpecSheet Schema
# ---------------------------------------

class SpecSheet(BaseModel):
    """
    Specification for HTML or Web Component rendering.
    """
    root: Component
    # metadata: Dict[str, Any] = {}            # Optional: author, timestamps, source LLM, etc.
    # datasets: Dict[str, JsonValue] = {}      # Optional: dynamic data made available to components

class Tags(Enum):
    DIV = "div"
    P = "p"
    SPAN = "span"
    BUTTON = "button"
    INPUT = "input"
    FORM = "form"
    TABLE = "table"
    TR = "tr"
    TD = "td"
    UL = "ul"
    LI = "li"
    H1 = "h1"
    H2 = "h2"
    H3 = "h3"
    H4 = "h4"
    H5 = "h5"
    H6 = "h6"
    IMG = "img"
    A = "a"

class HTMLSchema(BaseModel):
    """
    Specification for rendering standard HTML.
    """
    tag: Tags
    content: str                         # e.g., "div", "p", "span"

TypeError: unhashable type: 'dict'

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
) 

structured_llm = llm.with_structured_output(GenericJSON)
# ui_llm = llm.with_structured_output(SpecSheet)

In [79]:
query = "Give me step by step instructions to make a cup of tea."
# query = "compare nvidia gtx 1660 ti and nvidia rtx 3060 in terms of performance, price and power consumption"
# query = "names of 10 top movie directors along with their top movie from 2000 to 2005"

response = structured_llm.invoke(f"{query}.Return the response as a json object")

In [80]:
type(response.model_dump_json())

str

In [81]:
response.model_dump_json()

'{"data":"{\\"tea_instructions\\":[{\\"step\\":1,\\"instruction\\":\\"Boil water.\\"},{\\"step\\":2,\\"instruction\\":\\"Add a tea bag to a cup.\\"},{\\"step\\":3,\\"instruction\\":\\"Pour the hot water into the cup.\\"},{\\"step\\":4,\\"instruction\\":\\"Let the tea steep for a few minutes.\\"},{\\"step\\":5,\\"instruction\\":\\"Remove the tea bag.\\"},{\\"step\\":6,\\"instruction\\":\\"Add milk and sugar as desired.\\"}]}"}'

In [82]:
query = f"""
You are a UI Component Planner. You are responsible for generating SpecSheets based on provided data object in JSON format.
Return only the SpecSheet JSON without any additional explanations or comments. 
Make sure to return the json with 'root' as the top-level key.
Following is the data object you need to visualize:
{response.model_dump_json()}

Example SpecSheet Output:
{{
  "root": {{
    "type": "div",
    "children": [
      {{
        "type": "h1",
        "children": [{{ "type": "text", "value": "Title" }}]
      }},
      {{
        "type": "p",
        "children": [{{ "type": "text", "value": "This is a paragraph." }}]
      }},
      {{
      "table": {{
        "type": "table",
        "children": [
          {{
            "type": "tr",
            "children": [
              {{ "type": "th", "children": [{{ "type": "text", "value": "Column 1" }}] }},
              {{ "type": "th", "children": [{{ "type": "text", "value": "Column 2" }}] }}
            ]
          }},
          {{
            "type": "tr",
            "children": [
              {{ "type": "td", "children": [{{ "type": "text", "value": "Data 1" }}] }},
              {{ "type": "td", "children": [{{ "type": "text", "value": "Data 2" }}] }}
            ]
          }}
        ]
      }}
    ]
  }}
}}

"""
print(query)


You are a UI Component Planner. You are responsible for generating SpecSheets based on provided data object in JSON format.
Return only the SpecSheet JSON without any additional explanations or comments. 
Make sure to return the json with 'root' as the top-level key.
Following is the data object you need to visualize:
{"data":"{\"tea_instructions\":[{\"step\":1,\"instruction\":\"Boil water.\"},{\"step\":2,\"instruction\":\"Add a tea bag to a cup.\"},{\"step\":3,\"instruction\":\"Pour the hot water into the cup.\"},{\"step\":4,\"instruction\":\"Let the tea steep for a few minutes.\"},{\"step\":5,\"instruction\":\"Remove the tea bag.\"},{\"step\":6,\"instruction\":\"Add milk and sugar as desired.\"}]}"}

Example SpecSheet Output:
{
  "root": {
    "type": "div",
    "children": [
      {
        "type": "h1",
        "children": [{ "type": "text", "value": "Title" }]
      },
      {
        "type": "p",
        "children": [{ "type": "text", "value": "This is a paragraph." }]
      },


In [83]:
html_response = structured_llm.invoke(query)

In [84]:
html_response

GenericJSON(data='{"root":{"type":"div","children":[{"type":"h2","children":[{"type":"text","value":"Tea Instructions"}]},{"type":"ol","children":[{"type":"li","children":[{"type":"text","value":"Boil water."}]},{"type":"li","children":[{"type":"text","value":"Add a tea bag to a cup."}]},{"type":"li","children":[{"type":"text","value":"Pour the hot water into the cup."}]},{"type":"li","children":[{"type":"text","value":"Let the tea steep for a few minutes."}]},{"type":"li","children":[{"type":"text","value":"Remove the tea bag."}]},{"type":"li","children":[{"type":"text","value":"Add milk and sugar as desired."}]}]}]}}')

In [85]:
# print(html_response.model_dump())

In [86]:
json_obj = json.loads(html_response.data)
json_obj

{'root': {'type': 'div',
  'children': [{'type': 'h2',
    'children': [{'type': 'text', 'value': 'Tea Instructions'}]},
   {'type': 'ol',
    'children': [{'type': 'li',
      'children': [{'type': 'text', 'value': 'Boil water.'}]},
     {'type': 'li',
      'children': [{'type': 'text', 'value': 'Add a tea bag to a cup.'}]},
     {'type': 'li',
      'children': [{'type': 'text',
        'value': 'Pour the hot water into the cup.'}]},
     {'type': 'li',
      'children': [{'type': 'text',
        'value': 'Let the tea steep for a few minutes.'}]},
     {'type': 'li',
      'children': [{'type': 'text', 'value': 'Remove the tea bag.'}]},
     {'type': 'li',
      'children': [{'type': 'text',
        'value': 'Add milk and sugar as desired.'}]}]}]}}

In [87]:
def json_to_html(node):
    if node["type"] == "text":
        return node.get("value", "")
    tag = node["type"]
    children = node.get("children", [])
    inner_html = "".join(json_to_html(child) for child in children)
    return f"<{tag}>{inner_html}</{tag}>"

import json
# data = json.loads(json_obj)
html = json_to_html(json_obj["root"])
# print(html)

In [88]:
from IPython.display import display, HTML, JSON
display(HTML(html))